Imports

In [9]:
import feed_forward
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
torch.cuda.is_available()

True

CUDA check

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))])

In [14]:
trainset = datasets.CIFAR10('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.CIFAR10('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
# valloader = torch.utils.data.DataLoader(valset, batch_size=512, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in trainset if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in valset if label in [0, 2]]

Initializing the model

In [ ]:
model = feed_forward.HypFF(3072, 512, 256, 2)
model.to(device)